# **PTH FIXER FOR RVC MODELS**
## Applio has recently updated its version of PyTorch so recently trained models may not work on:
* Hugging Face Spaces (Like Ilaria RVC)
* W Okada Realtime Voice Changer
* Mainline or Mangio based Colab Notebooks

It appears that when the project transitioned to the new version of PyTorch, some key names in the dictionaries were altered. This change has rendered me unable to use a .pth file generated with Applio in the original RVC webUI.

### Credits to [LightningDragon](https://github.com/LightningDragon); [alexolotl](http://discord.com/users/718415427994124351) and [Not Eddy](http://discord.com/users/274566299349155851) staff of [AI HUB](https://discord.gg/aihub) server.

If you have any problem with this notebook you can ping [Not Eddy (Spanish Mod)](http://discord.com/users/274566299349155851)

In [ ]:
#@title # **1. Setup**
#@markdown Before running this cell create a folder called 'pthfix' in your GDrive and upload the .pth files you want to convert.
!pip install torch==2.1.1
from google.colab import drive
import os
import torch
from collections import OrderedDict
from IPython.display import clear_output
drive.mount('/content/drive')
main_folder_path = '/content/drive/MyDrive/pthfix'
fixed_folder_path = os.path.join(main_folder_path, 'Fixed')
clear_output()
if os.path.exists(main_folder_path):
    print("The pthfix folder already exists. You can continue.")
else:
    print("The pthfix folder has not been created by the user. Manually create the folder before continuing.")

In [ ]:
#@title # **2. Conversion**
os.chdir(main_folder_path)
if not os.path.exists(fixed_folder_path):
    os.makedirs(fixed_folder_path)
    print("The 'Fixed' folder has been created'")
else:
    print("The 'Fixed' folder already exists. OK!")
# LightningDragon code slightly modified
def replace_keys_in_dict(d, old_key_part, new_key_part):
    if isinstance(d, OrderedDict):
        updated_dict = OrderedDict()
    else:
        updated_dict = {}
    for key, value in d.items():
        if isinstance(key, str):
            new_key = key.replace(old_key_part, new_key_part)
        else:
            new_key = key
        if isinstance(value, dict):
            value = replace_keys_in_dict(value, old_key_part, new_key_part)
        updated_dict[new_key] = value
    return updated_dict
os.chdir(main_folder_path)
if not os.path.exists(fixed_folder_path):
    os.makedirs(fixed_folder_path)
    print("The 'Fixed' folder has been created")
else:
    print("The 'Fixed' folder already exists. OK!")
pth_files = [file for file in os.listdir(main_folder_path) if file.endswith('.pth') and not os.path.join(main_folder_path, file).startswith(fixed_folder_path)]
if len(pth_files) == 0:
    print("No .pth files found. Load at least one and run the cell again.")
else:
    print(f'Found .pth files: {len(pth_files)}')
    for pth_file in pth_files:
        pth_file_path = os.path.join(main_folder_path, pth_file)
        cgp_grey = torch.load(pth_file_path, map_location=torch.device('cpu'))
        processed_dict = replace_keys_in_dict(replace_keys_in_dict(cgp_grey, '.parametrizations.weight.original1', '.weight_v'), '.parametrizations.weight.original0', '.weight_g')
        new_file_name = pth_file.replace('.pth', '_fixed.pth')
        new_file_path = os.path.join(fixed_folder_path, new_file_name)
        torch.save(processed_dict, new_file_path)
        print(f"File processed and saved in 'Fixed' folder: {new_file_name}")